In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import argparse
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.quantized = False
        self.fc = nn.Linear(784, 10)
    def forward(self, x):
        if self.quantized:
            x = torch.quantize_per_tensor(x,0.02,51,torch.quint8)
        x = x.reshape(-1, 784)
        x = self.fc(x)
        if self.quantized:
            x = x.dequantize()
        return F.log_softmax(x, dim=1)
    def fuse_model(self):
        self.quantized =True

In [2]:
model = Net()
model.load_state_dict(torch.load('model28.pt'))

<All keys matched successfully>

In [3]:
batch_size = 10000
transform = transforms.Compose(
    [           transforms.ToTensor(),
    ])
test_dataset = torchvision.datasets.MNIST(root='./data/mnist/', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)
xs = []
ts = []
for batch_idx, (x, target) in enumerate(test_loader):
    xs.append(x)
    ts.append(target)

In [4]:
%%time
correct_cnt, ave_loss = 0, 0
model.eval()
for x,target in zip(xs,ts):
    out = model(x)
    _, pred_label = torch.max(out.data, 1)
    correct_cnt += (pred_label == target.data).sum()

print(correct_cnt)

tensor(9250)
CPU times: user 0 ns, sys: 5.05 ms, total: 5.05 ms
Wall time: 2.57 ms


In [5]:
qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
model.qconfig = qconfig
torch.quantization.prepare(model, inplace=True)
for x,target in zip(xs,ts):
    out = model(x)
torch.quantization.convert(model, inplace=True)
model.fuse_model()

In [6]:
%%time
correct_cnt, ave_loss = 0, 0
model.eval()
for x,target in zip(xs,ts):
    out = model(x)
    _, pred_label = torch.max(out.data, 1)
    correct_cnt += (pred_label == target.data).sum()

print(correct_cnt)

tensor(9249)
CPU times: user 18.7 ms, sys: 5.23 ms, total: 23.9 ms
Wall time: 3.78 ms
